In [7]:
constraints=["111 112 113 114 115 116 117 118 119 0",
"-111 -112 0",
"-111 -113 0",
"-111 -114 0",
"-111 -115 0",
"-111 -116 0",
"-111 -117 0",
"-111 -118 0",
]

In [42]:
constraintDict={}
literalsDict={}
i=0
for clause in constraints:
    if i not in constraintDict.keys():
        constraintDict[i]={}
        
    for literal in clause.split():
        lit=int(literal)
        if lit!=0:
            constraintDict[i][lit]=False
            if lit not in literalsDict.keys():
                literalsDict[lit]=[i]
            else:
                literalsDict[lit].append(i)
    i+=1
print(constraintDict)
# print(literalsDict)

{0: {111: False, 112: False, 113: False, 114: False, 115: False, 116: False, 117: False, 118: False, 119: False}, 1: {-111: False, -112: False}, 2: {-111: False, -113: False}, 3: {-111: False, -114: False}, 4: {-111: False, -115: False}, 5: {-111: False, -116: False}, 6: {-111: False, -117: False}, 7: {-111: False, -118: False}}


In [ ]:
assignments={}
def DP(constraints):
    if len(constraints)==0:
#         print("satisfiable solution")
        return True
   
    if emptyClause(constraints):
#         print("no satisfiability")
        return False
    #maybe do it one time beforehand
    hasFound,constraints=tautology(constraints):
    if hasFound:
        DP(constraints)
        
    hasFound,constraints=unitCl(constraints)
    if hasFound:
        DP(constraints)
        
    hasFound,constraints=pureLit(constraints)
    if hasFound:
        DP(constraints)
                
    reducedConstraints=pickRandomLiteral(constraints)
    if DP(reducedConstraints)==True:
#         print("satisfiable solution")
        return True
    else:
        DP(constraints)

        

In [22]:
def emptyClause(constraints):
    for clause in constraints.keys():
        if not constraints[clause]:
            return True
    return False

In [35]:
def tautology(constraints):
    foundOne=False
    for clause in constraints.keys():
        literals=list(constraints[clause].keys())
        print(literals)
        counterLiterals=[-x for x in literals]
        
        for i in literals:
            for j in counterLiterals:
            if i==j:
                del constraints[clause]
                foundOne=True
    return foundOne,constraints
        

In [ ]:
def unitCl(constraints,literalsDict):
    literals=[]
    for clause in constraints.keys():
        if len(constraints[clause].keys()==1):
            literals.append(constraints[clause])
            del constraints[clause]
    for literal in literals:
        clauses=literalsDict[literal]
        for clause in clauses:
            del constraintsDict[clause]
    counterLiterals=[-x for x in literals]
    for literal in counterLiterals:
        clauses=literalsDict[literals]
        for clause in clauses:
            del constraintsDict[clause][literal]
    
    if len(units)>0: 
        return True,constraints
    else:
        return False,constraints

In [ ]:
def pureLit(constraints,litDict):
    foundOne=False
    literals=list(litDict.keys())
    doubleLiterals=[]
    for i in literals:
            for j in counterLiterals:
                if i==j:
                    doubleLiterals.append(i)
    singelLiterals=[x for x in literals if x not in doubleLiterals]
    clauses=[litDict[x] for x in singelLiterals]   
    
    for clause in clauses:
        foundOne=True
        del constraints[i]
        
    for literal in singleLiteral:
        clauses=literalsDict[literal]
            for clause in clauses:
                del constraints[clause][literal]
    return foundOne,constraints
        

In [ ]:
def pickRandomLiteral(constraints)